# P&G CEO Challenge 1st round

This project is our implementation of the idea for a competition, although it was really a pity failing to break into semi-round, it has been really an honor to work with the brilliant teammates. Due to the lack of data support, we choose to use a public dataset similar to the data we expected, turn the blueprint into reality and draw a conclusion to this journey:)
Due to the rules of the competition, we will not disclose any details of our slides, but only try to illustrate the feasibility and some important results, the dataset contains all the transactions occurring for a UK-based and registered, non-store online retail between 01/12/2009 and 09/12/2011.  http://archive.ics.uci.edu/ml/datasets/Online+Retail+II 

## Environment Setup

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Getting and Cleaning Data

There are over 500k records, pretty large a dataframe.

In [0]:
df = pd.read_csv('data.csv', encoding="ISO-8859-1")
df.shape

(541909, 8)

There are 8 attributes, "InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID" and "Country". Among them, InvoiceNo is a 6-digit integral number uniquely assigned to each transaction; StockCode is a 5-digit integral number uniquely assigned to each distinct product. 

In [0]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


There are a small portion of Description recorded as NaN and nearly a quarter of CustomerID missing information, since we have such a great volume of data and this part of data is extremely hard to impute, we discard them.

In [0]:
df_NAN = df.isna().sum()
df_NAN = df_NAN.to_frame()
df_NAN.columns = ['Count']

df_NAN_percentage = df.isna().sum()/len(df) * 100
df_NAN_percentage = df_NAN_percentage.to_frame()
df_NAN_percentage.columns = ['Percentage']

df_NAN.merge(df_NAN_percentage, left_index=True, right_index=True)

,Count,Percentage
InvoiceNo,0,0.000000
StockCode,0,0.000000
Description,1454,0.268311
Quantity,0,0.000000
InvoiceDate,0,0.000000
UnitPrice,0,0.000000
CustomerID,135080,24.926694
Country,0,0.000000


In [0]:
#Discard records without Description and CustomerID
df = df.dropna(subset=['Description', 'CustomerID'])
df.shape

(406829, 8)

In [0]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

Transform datatype of 'CustomerID' into str, since it only represents the identity code of a unique customer; transform 'InvoiceDate' into datetime.

In [0]:
df['CustomerID'] = df['CustomerID'].astype(int).astype(str)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
Country                object
dtype: object

In [0]:
df_NAN = df.isna().sum()
df_NAN = df_NAN.to_frame()
df_NAN.columns = ['Count']

df_NAN_percentage = df.isna().sum()/len(df) * 100
df_NAN_percentage = df_NAN_percentage.to_frame()
df_NAN_percentage.columns = ['Percentage']

df_NAN.merge(df_NAN_percentage, left_index=True, right_index=True)

,Count,Percentage
InvoiceNo,0,0.0
StockCode,0,0.0
Description,0,0.0
Quantity,0,0.0
InvoiceDate,0,0.0
UnitPrice,0,0.0
CustomerID,0,0.0
Country,0,0.0


After dropping useless rows, there still leaves over 400k records, then we find the duplicate records and further reduce the data volume. 

In [0]:
df.duplicated().sum()

5225

In [0]:
df = df.drop_duplicates()
df.shape

(401604, 8)

## Exploratory Data Analysis

It is werid that there is negative value for 'Quantity', the common patern for these negative rows is a prefix 'C' of their 'InvoiceNo', which represents 'Cancelation'. Some of rows have corresponding transactions of which the 'Quantity' is exactly the absolute value, we discard both rows as it will not reflect the willingness to purchase; while some are without corresponding purchasing records, probably because the purchase was done before 01/12/2009.

In [0]:
df.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,401604,401604,401604,401604.000000,401604,401604.000000,401604,401604
unique,22190,3684,3896,NaN,20460,NaN,4372,37
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-11-14 15:27:00,NaN,17841,United Kingdom
freq,542,2065,2058,NaN,543,NaN,7812,356728
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,12.183273,NaN,3.474064,NaN,NaN
std,NaN,NaN,NaN,250.283037,NaN,69.764035,NaN,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,0.000000,NaN,NaN
25%,NaN,NaN,NaN,2.000000,NaN,1.250000,NaN,NaN


In [0]:
InvoiceNo = df.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
plt.plot(InvoiceNo['CustomerID'], InvoiceNo['InvoiceNo'])
plt.show()

In [0]:
#Select attributes 'InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate' and 'CustomerID' for comparison with purchase records
df_cancel = df[df['Quantity'] < 0][['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'CustomerID']]
df_cancel

,InvoiceNo,StockCode,Quantity,InvoiceDate,CustomerID
141,C536379,D,-1,2010-12-01 09:41:00,14527.0
154,C536383,35004C,-1,2010-12-01 09:49:00,15311.0
235,C536391,22556,-12,2010-12-01 10:24:00,17548.0
236,C536391,21984,-24,2010-12-01 10:24:00,17548.0
237,C536391,21983,-24,2010-12-01 10:24:00,17548.0
...,...,...,...,...,...
540449,C581490,23144,-11,2011-12-09 09:57:00,14397.0
541541,C581499,M,-1,2011-12-09 10:28:00,15498.0
541715,C581568,21258,-5,2011-12-09 11:57:00,15311.0
541716,C581569,84978,-1,2011-12-09 11:58:00,17315.0


In [0]:
A = [] #Cancelation with corresponding purchase record
B = [] #Cancelation without corresponding purchase record
index_cor = [] #Index of corresponding records
for index, attributes in df_cancel.iterrows():
    df_cor = df[(df['StockCode'] == attributes[1]) & (df['Quantity'] == -attributes[2]) & (df['CustomerID'] == attributes[3])]
    if df_cor.shape[0] == 0:
        B.append(index)
    else:
        A.append(index)
        index_cor.append(df_cor.index)

In [0]:
print('Cancelation with corresponding purchase record: {}'.format(len(A)))
print('Index of corresponding records: {}'.format(len(index_cor)))
print('Cancelation without corresponding purchase record: {}'.format(len(B)))

Cancelation with corresponding purchase record: 0
Index of corresponding records: 0
Cancelation without corresponding purchase record: 8872


In [0]:
#Delete cancelation records
df = df[~df.index.isin(A)]
df = df[~df.index.isin(B)]
df.shape

(392732, 8)

In [0]:
df.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,392732,392732,392732,392732.000000,392732,392732.000000,392732,392732
unique,18536,3665,3877,NaN,17286,NaN,4339,37
top,576339,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,2011-11-14 15:27:00,NaN,17841.0,United Kingdom
freq,542,2023,2016,NaN,542,NaN,7676,349227
first,NaN,NaN,NaN,NaN,2010-12-01 08:26:00,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,2011-12-09 12:50:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,13.153718,NaN,3.125596,NaN,NaN
std,NaN,NaN,NaN,181.588420,NaN,22.240725,NaN,NaN
min,NaN,NaN,NaN,1.000000,NaN,0.000000,NaN,NaN
25%,NaN,NaN,NaN,2.000000,NaN,1.250000,NaN,NaN


In [0]:
print('Number of transactions: {}'.format(len(df['InvoiceNo'].value_counts())))
print('Number of products: {}'.format(len(df['StockCode'].value_counts())))
print('Number of customers: {}'.format(len(df['CustomerID'].value_counts())))

Number of transactions: 18536
Number of products: 3665
Number of customers: 4339
